# Generative Search with xAI

In this demo, we will use Weaviate's Embedding Service to generate embeddings for the blog posts and a generative model on xAI to create new content!

## Requirements

1. Serverless Weaviate cluster (for the Weaviate Embedding Service)
    1. You can create a 14-day free sandbox on [WCD](https://console.weaviate.cloud/)
2. xAI API key. Grab one [here](https://docs.x.ai/docs/overview).

In [1]:
import weaviate
from weaviate.embedded import EmbeddedOptions
import weaviate.classes as wvc
import weaviate.classes.config as wc
import requests, json
import weaviate.classes.query as wq
from weaviate.classes.config import Property, DataType
import os
import re
from weaviate.util import get_valid_uuid
from uuid import uuid4

## Connect to Weaviate

Only choose one option from the below.

**Weaviate Cloud Deployment**

In [3]:
WCD_URL = os.environ["WEAVIATE_URL"] # Replace with your Weaviate cluster URL
WCD_AUTH_KEY = os.environ["WEAVIATE_AUTH"] # Replace with your cluster auth key
XAI_KEY = os.environ["XAI_API_KEY"] # Replace with your xAI key

# Weaviate Cloud Deployment
client = weaviate.connect_to_wcs(
    cluster_url=WCD_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WCD_AUTH_KEY),
      headers={ "X-Xai-Api-Key": XAI_KEY}
)

print(client.is_ready())

True


## Create a collection
Collection stores your data and vector embeddings.

Full list of [generative models](https://weaviate.io/developers/weaviate/model-providers/octoai/generative#available-models)

In [4]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "BlogChunks", and then you need to re-import it again.

# Delete the collection if it already exists
if (client.collections.exists("BlogChunks")):
    client.collections.delete("BlogChunks")

client.collections.create(
    name="BlogChunks",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_weaviate( # specify the vectorizer and model type you're using
        model="Snowflake/snowflake-arctic-embed-l-v2.0", # default model
    ),

    generative_config=wc.Configure.Generative.xai( 
        model="grok-2-latest" # select model, default is grok-2-latest
    ),

    properties=[
            Property(name="content", data_type=DataType.TEXT) # We only have one property for our collection. It is the content within the blog posts
    ]
)

print("Successfully created collection: BlogChunks.")

Successfully created collection: BlogChunks.


## Chunk and Import Data

We need to break our blog posts into smaller chunks

In [5]:
def chunk_list(lst, chunk_size):
    """Break a list into chunks of the specified size."""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

def split_into_sentences(text):
    """Split text into sentences using regular expressions."""
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

def read_and_chunk_index_files(main_folder_path):
    """Read index.md files from subfolders, split into sentences, and chunk every 5 sentences."""
    blog_chunks = []
    
    for file_path in os.listdir("./data"):
        index_file_path = os.path.join("./data", file_path)
        with open(index_file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            sentences = split_into_sentences(content)
            sentence_chunks = chunk_list(sentences, 5)
            sentence_chunks = [' '.join(chunk) for chunk in sentence_chunks]
            blog_chunks.extend(sentence_chunks)
    return blog_chunks

# run with:
main_folder_path = './data'
blog_chunks = read_and_chunk_index_files(main_folder_path)

In [6]:
# First chunk

blog_chunks[0]

'---\ntitle: What is Ref2Vec and why you need it for your recommendation system\nslug: ref2vec-centroid\nauthors: [connor]\ndate: 2022-11-23\ntags: [\'integrations\', \'concepts\']\nimage: ./img/hero.png\ndescription: "Weaviate introduces Ref2Vec, a new module that utilises Cross-References for Recommendation!"\n---\n![Ref2vec-centroid](./img/hero.png)\n\n<!-- truncate -->\n\nWeaviate 1.16 introduced the [Ref2Vec](/developers/weaviate/modules/retriever-vectorizer-modules/ref2vec-centroid) module. In this article, we give you an overview of what Ref2Vec is and some examples in which it can add value such as recommendations or representing long objects. ## What is Ref2Vec? The name Ref2Vec is short for reference-to-vector, and it offers the ability to vectorize a data object with its cross-references to other objects. The Ref2Vec module currently holds the name ref2vec-**centroid** because it uses the average, or centroid vector, of the cross-referenced vectors to represent the **referen

In [7]:
# Insert the objects (chunks) into the Weaviate cluster

blogs = client.collections.get("BlogChunks")

for blog_chunk in blog_chunks:
    random_uuid = get_valid_uuid(uuid4())
    blogs.data.insert(
        properties={
            "content": blog_chunk
        },
        uuid=random_uuid
    )


## Query Time 

## Hybrid Search Query

Hybrid search combines BM25 and vector search and weighs the two algorithms depending on the `alpha` parameter. 

`alpha`= 0 --> pure BM25

`alpha`= 0.5 --> half BM25, half vector search

`alpha`= 1 --> pure vector search

In [8]:
import json 

blogs = client.collections.get("BlogChunks")

response = blogs.query.hybrid(
    query="What is Ref2Vec",
    alpha=0.5,
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "content": "---\ntitle: What is Ref2Vec and why you need it for your recommendation system\nslug: ref2vec-centroid\nauthors: [connor]\ndate: 2022-11-23\ntags: ['integrations', 'concepts']\nimage: ./img/hero.png\ndescription: \"Weaviate introduces Ref2Vec, a new module that utilises Cross-References for Recommendation!\"\n---\n![Ref2vec-centroid](./img/hero.png)\n\n<!-- truncate -->\n\nWeaviate 1.16 introduced the [Ref2Vec](/developers/weaviate/modules/retriever-vectorizer-modules/ref2vec-centroid) module. In this article, we give you an overview of what Ref2Vec is and some examples in which it can add value such as recommendations or representing long objects. ## What is Ref2Vec? The name Ref2Vec is short for reference-to-vector, and it offers the ability to vectorize a data object with its cross-references to other objects. The Ref2Vec module currently holds the name ref2vec-**centroid** because it uses the average, or centroid vector, of the cross-referenced vectors to represent 

### Generative Search Query

Here is what happens in the below:
1. We will retrieve 3 relevant chunks from our vector database
2. We will pass the 3 chunks to xAI to generate the short paragraph about Ref2Vec

The first line in the output is the generated text, and the `content` pieces below it, are what was retrieved from Weaviate and passed to xAI.

In [9]:
blogs = client.collections.get("BlogChunks")


response = blogs.generate.near_text(
    query="What is Ref2Vec?",
    single_prompt="Write a short paragraph about ref2vec with this content: {content}",
    limit=3
)


for o in response.objects:
    print(o.generated)
    print(json.dumps(o.properties, indent=2))

Here is a short paragraph about Ref2Vec:

Ref2Vec, short for reference-to-vector, is a module in Weaviate 1.16 that enables the vectorization of a data object by incorporating its cross-references to other objects. The Ref2Vec module, specifically ref2vec-centroid, generates a representation of the referencing object by using the average, or centroid vector, of the cross-referenced vectors. This innovative approach can add significant value to applications such as recommendation systems and representing long objects.
{
  "content": "---\ntitle: What is Ref2Vec and why you need it for your recommendation system\nslug: ref2vec-centroid\nauthors: [connor]\ndate: 2022-11-23\ntags: ['integrations', 'concepts']\nimage: ./img/hero.png\ndescription: \"Weaviate introduces Ref2Vec, a new module that utilises Cross-References for Recommendation!\"\n---\n![Ref2vec-centroid](./img/hero.png)\n\n<!-- truncate -->\n\nWeaviate 1.16 introduced the [Ref2Vec](/developers/weaviate/modules/retriever-vectori